In [ ]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
import time

In [ ]:

driver_path = "/usr/local/bin/chromedriver"

main_url = "https://www.hepsiburada.com/"

# Create an empty DataFrame for user reviews and products

reviews_df = pd.DataFrame(columns=['user reviews', 'rating', 'product url'])

products_df = pd.DataFrame(columns=['product url', 'product name', 'img url','brand name','category name'])

service = ChromeService(executable_path=driver_path)

driver = webdriver.Chrome(service=service)

driver.get(main_url)

# Get the category name

action = ActionChains(driver)

navbar_element = driver.find_element(By.CLASS_NAME,'sf-MenuItems-WulWXvlfIAwNiOUGY7FP') 

action.move_to_element(navbar_element).perform()

time.sleep(4)

category_tag = driver.find_element(By.CLASS_NAME,'sf-ChildMenuItems-a4G0z3YJJWkQs7qKKuuj')

category_name = category_tag.text

category_tag.click()

# Find <li> tags containing the list of products

li_tags = driver.find_elements(By.CLASS_NAME,"productListContent-zAP0Y5msy8OHn5z7T_K_")
                                            
# Loop through each product

for li in li_tags:
    product_url = None
    product_name = None
    img_url = None
    user_review = None
    rating = None
    brand_name = None
    
    # Get the product URL

    product_url = li.find_element(By.TAG_NAME,"a").get_attribute("href")

    # Open a new window, Switch to the new window

    driver.execute_script("window.open('');")

    driver.switch_to.window(driver.window_handles[-1]) 

    driver.get(product_url)

    driver.set_window_size(1200, 900)

    # Get the product name

    product_name = driver.find_element(By.ID,"product-name").text

    # Get the brand name 

    brand_name = driver.find_element(By.XPATH,"/html/body/div[2]/main/div[3]/section[1]/div[3]/div/div[4]/div[1]/span/a").text

    # Get the product image URL

    img_url = driver.find_element(By.CSS_SELECTOR,"img[class='product-image']").get_attribute("src")

    # Scroll down the page, Wait for user reviews to load

    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

    driver.find_element(By.XPATH,'//*[@id="productReviewsTab"]').click()

    # Process each user review

    try:

        all_reviews_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='hermes-voltran-comments']/div[2]/div[1]/a"))).click()

    except TimeoutException:

        driver.close()

        # Switch back to the main window

        driver.switch_to.window(driver.window_handles[0])
        continue

    
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

    last_reviews = driver.find_elements(By.XPATH,'//*[@id="hermes-voltran-comments"]/div[6]/div[3]/div[2]/div/ul/li')[-1].text
    
    for i in range(int(last_reviews)):

        all_reviews_div = driver.find_elements(By.CLASS_NAME,"hermes-ReviewCard-module-BJtQZy5Ub3goN_D0yNOP")
                     
        for review in all_reviews_div:

            rating = len(review.find_elements(By.CSS_SELECTOR, "div[class='star']"))
        
            try:
                user_review = review.find_element(By.CSS_SELECTOR, "span[itemprop='description']").text

            except:
                user_review = None
            
            reviews_df.loc[len(reviews_df)] = [user_review,rating,product_url]

        driver.set_window_size(400, 826)

        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        next_page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="hermes-voltran-comments"]/div[6]/div[3]/div[2]/div/div/div/div[3]')))

        driver.execute_script("arguments[0].click()",next_page_button)
                        
    driver.close()

    # Switch back to the main window

    driver.switch_to.window(driver.window_handles[0])

    # Add products to the DataFrame

    products_df.loc[len(products_df)]=[product_url,product_name,img_url,brand_name,category_name]

driver.quit()

In [113]:
products_df

,product url,product name,img url,brand name,category name
0,https://www.hepsiburada.com/lenovo-legion-oyun...,Lenovo Legion Oyuncu Mouse Pad GXY0K07130,https://productimages.hepsiburada.net/s/38/550...,Lenovo,Bilgisayar/Tablet
1,https://www.hepsiburada.com/lenovo-400-kablosu...,Lenovo 400 Kablosuz Mouse GY50R91293,https://productimages.hepsiburada.net/s/31/550...,Lenovo,Bilgisayar/Tablet
2,https://adservice.hepsiburada.com/event/api/v1...,Hp 305XL Renkli Yüksek Kapasiteli Mürekkep Kar...,https://productimages.hepsiburada.net/s/43/550...,HP,Bilgisayar/Tablet


In [114]:
reviews_df

,user reviews,rating,product url
0,"Tam bir fiyat performans ürünü. Yüzeyi kaygan,...",5,https://www.hepsiburada.com/lenovo-legion-oyun...
1,"F/P Bir Mouse Pad, Lenovo kalitesi.",5,https://www.hepsiburada.com/lenovo-legion-oyun...
2,Yorumlarda yazdığı gibi büyük olan bi yanı yok...,4,https://www.hepsiburada.com/lenovo-legion-oyun...
3,Kaliteli.,5,https://www.hepsiburada.com/lenovo-legion-oyun...
4,None,5,https://www.hepsiburada.com/lenovo-legion-oyun...
...,...,...,...
7225,ürünü alır almaz yorum yapmayı sevmem bu ürünü...,1,https://www.hepsiburada.com/logitech-m220-sess...
7226,Ürün hem sahte hem eksik geldi. Detaylı incele...,1,https://www.hepsiburada.com/logitech-m220-sess...
7227,SimayTech isimli mağazadan iki adet almıştım. ...,1,https://www.hepsiburada.com/logitech-m220-sess...
7228,Satıcısının hepsi burada olmasina rağmen ürün ...,1,https://www.hepsiburada.com/logitech-m220-sess...


In [58]:
products_df.to_csv('products_df.csv',index=False,encoding="utf-8")

In [59]:
reviews_df.to_csv('reviews_df.csv',index=False,encoding="utf-8")